In [1]:
library(plyr)
library(tidyverse)
library(infer)
library(janitor)
library(repr)
library(stringr)
library(ggthemes)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [5]:
# url <- "https://geodash.vpd.ca/opendata/crimedata_download/crimedata_csv_all_years.zip"
# filename <- "crime_data.zip"
# download.file(url, destfile = filename)

# # Data comes in as a zip, so we'll need to extract it
# unzip("crime_data.zip")

# Read in the desired file
crime_data <- read_csv("crimedata_csv_all_years.csv")



── Column specification ────────────────────────────────────────────────────────
cols(
  TYPE = col_character(),
  YEAR = col_double(),
  MONTH = col_double(),
  DAY = col_double(),
  HOUR = col_double(),
  MINUTE = col_double(),
  HUNDRED_BLOCK = col_character(),
  NEIGHBOURHOOD = col_character(),
  X = col_double(),
  Y = col_double()
)




In [18]:
theft_crimes <- c("Other Theft", "Theft from Vehicle", 
                  "Theft of Bicycle", "Theft of Vehicle")
crime_data <- crime_data %>% clean_names() %>% mutate(is_theft = type%in% theft_crimes)

In [19]:
pre_years <- c(2017, 2018, 2019)


In [20]:
neighbourhoods <- unique(crime_data$neighbourhood)
neighbourhoods <- tibble(neighbourhood=as.character(neighbourhoods[!is.na(neighbourhoods)]))
columns <- c("neighbourhood", "past_prop", "current_prop", "diff_in_props", "p_value")

In [21]:
crime_data_processed <- crime_data %>%
    select(neighbourhood, year, is_theft) %>%
    filter(year >= 2017 & year < 2021) %>%
    mutate(period=ifelse(year %in% pre_years, 'pre_covid', 'covid')) %>%
    mutate(period=as.factor(period)) %>%
    mutate(neighbourhood=neighbourhood)
head(crime_data_processed)

neighbourhood,year,is_theft,period
<chr>,<dbl>,<lgl>,<fct>
Fairview,2019,FALSE,pre_covid
West End,2019,FALSE,pre_covid
West End,2020,FALSE,covid
West End,2017,FALSE,pre_covid
West End,2018,FALSE,pre_covid
West End,2020,FALSE,covid


In [22]:
results <- data.frame(matrix(ncol=length(columns), nrow=0))
names(results) <- columns

In [23]:
set.seed(12345)
for (i in 1:nrow(neighbourhoods)) {
    n <- neighbourhoods$neighbourhood[i]
    neighbourhood_data <- crime_data_processed %>% filter(neighbourhood==n)
    
    pre_covid <- neighbourhood_data %>%
        filter(period=="pre_covid")
    pre_covid_prop <- mean(as.numeric(pre_covid$is_theft))
    
    covid <- neighbourhood_data %>%
        filter(period=="covid")
    covid_prop <- mean(as.numeric(covid$is_theft))
    
    obs_diff_in_props <- covid_prop - pre_covid_prop
    
    neighbourhood_infer <- neighbourhood_data %>%
        specify(is_theft ~ period, success="TRUE") %>%
        hypothesise(null="independence") %>%
        generate(reps=1000, type="permute") %>%
        calculate(stat="diff in props", order=c("covid", "pre_covid"))
    
    p_value <- neighbourhood_infer %>% get_p_value(obs_diff_in_props, direction="two_sided") %>% pull()
    
    new_obs <- data.frame(n, pre_covid_prop, covid_prop, obs_diff_in_props, p_value)
    names(new_obs) <- columns
    results <- rbind(results, new_obs)
}

Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()

In [24]:
results <- results %>%
    mutate(reject5=p_value < 0.05, reject10=p_value < 0.10)
results

neighbourhood,past_prop,current_prop,diff_in_props,p_value,reject5,reject10
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
Oakridge,0.5432873,0.4968354,-0.046451884,0.176,FALSE,FALSE
Fairview,0.6891537,0.6194740,-0.069679692,0.000,TRUE,TRUE
West End,0.7351588,0.6331310,-0.102027854,0.000,TRUE,TRUE
Central Business District,0.7533982,0.6046934,-0.148704859,0.000,TRUE,TRUE
Hastings-Sunrise,0.6451613,0.5693642,-0.075797128,0.000,TRUE,TRUE
Strathcona,0.5577118,0.4385246,-0.119187188,0.000,TRUE,TRUE
Grandview-Woodland,0.6237392,0.5506849,-0.073054262,0.000,TRUE,TRUE
Kitsilano,0.6463512,0.6171575,-0.029193753,0.052,FALSE,TRUE
Kensington-Cedar Cottage,0.6043016,0.6161616,0.011860044,0.444,FALSE,FALSE
